#### **Import Libraries**

In [4]:
import torch
import transformers 
from IPython.display import display
from IPython.display import Markdown
import textwrap
from transformers import AutoTokenizer, AutoModelForCausalLM
torch.cuda.is_available()

True

#### **Model ID**

In [3]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

#### **Tokenizer**

In [6]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

terminators = [
    tokenizer.eos_token_id, 
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### **Pipeline**

In [5]:
pipeline = transformers.pipeline(
    'text-generation',
    model=model_id, 
    model_kwargs={"torch_dtype": torch.bfloat16,
                  "quantization_config": {"load_in_4bit": True}, 
                  "low_cpu_mem_usage": True}
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
def get_prompt(desc):
    return f"""Task: The following is a description of an eviction case, predict whether the tenant has legal represenation 

     Content: {desc}

     Prediction:"""

In [10]:
desc = "No, the Right to Counsel is not in effect in the tenant's zip code. This is a summary process (eviction) complaint filed in the Superior Court of Connecticut. The plaintiff, who is the landlord, is seeking to terminate the lease agreement with the defendant, who is the tenant, due to the lapse of time. The lease agreement, which was either oral or in writing, was entered into on January 1, 2015, and the defendant agreed to rent the premises located at 1 Davies Court (Basement and Common Area), Ansonia, CT XXXXX. The defendant agreed to pay $400 weekly or monthly on the 1st day of each week or month.\n\nThe plaintiff alleges that the defendant has used and occupied the premises as agreed under the lease and still occupies the premises. However, the lease has terminated by lapse of time. On March 8, 2022, the plaintiff served a Notice to Quit (End) Possession on the defendant, requiring them to move out of the premises on or before March 31, 2022. The time given in the notice has passed, but the defendant has not moved out.\n\nThe plaintiff is seeking judgment for immediate possession of the premises and also requests forfeiture of the defendant's possessions and personal effects because this is a nonresidential property. The complaint is signed by the plaintiff or their attorney on March 31, 2022."

In [11]:
messages = [
    {"role": "system", "content": "You are a housing court clerk"}, 
    {"role": "user", "content": get_prompt(desc)}]

prompt = pipeline.tokenizer.apply_chat_template(
    messages, 
    tokenize=False, 
    add_generation_prompt=True
)

In [8]:
### ---         Print Markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))
### ---

In [12]:
outputs = pipeline(
    prompt, 
    max_new_tokens=256, 
    eos_token_id=terminators, 
    do_sample=True, 
    temperature=0.1,
    top_p=0.9
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [13]:
to_markdown(outputs[0]["generated_text"][len(prompt):])

> Based on the information provided, I predict that the tenant does not have legal representation. The case is a summary process eviction, which is a simpler and faster process compared to a regular eviction case. The fact that the plaintiff is seeking judgment for immediate possession of the premises and forfeiture of the defendant's possessions and personal effects suggests that this is a straightforward case with no complex legal issues involved. Additionally, the fact that the plaintiff is seeking to terminate the lease agreement due to the lapse of time implies that the tenant's defenses may be limited, making it less likely that they would need legal representation.

: 